# Danny's Diner Case Study
#### Avery Lee 

8 Week SQL Challenge #1: https://8weeksqlchallenge.com/case-study-1/

![Intro Image](images/intro_image.png)

## Table of Contents 
- Problem Statement
- The Datasets
    - `sales` table
    - `members` table
    - `menu` table
- Case Study Questions
    - 1. What is the total amount each customer spent at the restaurant?
    - 2. How many days has each customer visited the restaurant?
    - 3. What was the first item from the menu purchased by each customer?
    - 4. What is the most purchased item on the menu and how many times was it purchased by all customers?
    - 5. Which item was the most popular for each customer?
    - 6. Which item was purchased first by the customer after they became a member?
    - 7. Which item was purchased just before the customer became a member?
    - 8. What is the total items and amount spent for each member before they became a member?
    - 9. If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?
    - 10. In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?
- Bonus Tasks
    - 1. Create a table that contains `customer_id`, `order_date`, `product_name`, `price`, and `member` which indicates 'Y' if a customer is a member on that date and 'N' if they are not.
    - 2. Using the saved table above, also add `ranking` which indicates the order of each item for a customer after they have joined the loyalty program.

## Problem Statement 
Danny opens up a restaurant that sells his 3 favourite foods: sushi, curry, and ramen. He wants to use the data to answer a few simple questions about his customers, including their visitng patterns, how much money they have spent, and which menu items are their favorite. He plans on using these insights to help him decide whether he should expand the existing customer loyalty program - additionally he needs help to generate some basic datasets so his team can easily inspect the data without needing to use SQL.

We will answer the following questions: 

1. What is the total amount each customer spent at the restaurant?
2. How many days has each customer visited the restaurant?
3. What was the first item from the menu purchased by each customer?
4. What is the most purchased item on the menu and how many times was it purchased by all customers?
5. Which item was the most popular for each customer?
6. Which item was purchased first by the customer after they became a member?
7. Which item was purchased just before the customer became a member?
8. What is the total items and amount spent for each member before they became a member?
9. If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?
10. In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?

Bonus task: Create tables that will be useful for Danny to quickly derive insights. 

## The Datasets

In [2]:
# Load the SQL extension
%load_ext sql

# Connect
%sql sqlite:////Users/averylee/Desktop/DS/SQL/dannys_diner/dannys_diner.db

This is the entity relationship diagram provided by the case study. The `customer_id` and `product_id` from the `sales` table respectively correspond to the same in the `members` and `menu` tables. 

![ERD Image](images/entity_relationship_diagram.png)

#### `sales` Table 

This table consists of multiple customers and their unique `customer_id`, `order_date`, and the item they ordered which is represented by the `product_id`.

In [3]:
%%sql
SELECT * FROM sales

 * sqlite:////Users/averylee/Desktop/DS/SQL/dannys_diner/dannys_diner.db
Done.


customer_id,order_date,product_id
A,2021-01-01,1
A,2021-01-01,2
A,2021-01-07,2
A,2021-01-10,3
A,2021-01-11,3
A,2021-01-11,3
B,2021-01-01,2
B,2021-01-02,2
B,2021-01-04,1
B,2021-01-11,1


#### `members` Table 

This table contains the `join_date` a `customer_id` joined the beta version of Danny's loyalty program. 

In [4]:
%%sql
SELECT * FROM members

 * sqlite:////Users/averylee/Desktop/DS/SQL/dannys_diner/dannys_diner.db
Done.


customer_id,join_date
A,2021-01-07
B,2021-01-09


#### `menu` Table

This table contains the `product_name` and `price` for each `product_id`.

In [5]:
%%sql
SELECT * FROM menu

 * sqlite:////Users/averylee/Desktop/DS/SQL/dannys_diner/dannys_diner.db
Done.


product_id,product_name,price
1,sushi,10
2,curry,15
3,ramen,12


## Case Study Questions 

<a id="question-1"></a>
#### 1. What is the total amount each customer spent at the restaurant?

Steps: 
- JOIN the `sales` and `menu` tables using the `product_id` column to get the corresponding price for each menu item the customer purchased. 
- SUM up the `price` each customer puchased. 
- ORDER BY the `customer_id` for a clean output. 

Results: 
- Customer A spent $76. 
- Customer B spent $74. 
- Customer C spent $36. 

In [6]:
%%sql 

select s.customer_id, sum(m.price) as price
from sales s join menu m on s.product_id = m.product_id
group by s.customer_id
order by s.customer_id

 * sqlite:////Users/averylee/Desktop/DS/SQL/dannys_diner/dannys_diner.db
Done.


customer_id,price
A,76
B,74
C,36


<a id="question-2"></a>
#### 2. How many days has each customer visited the restaurant?

Steps: 
- COUNT the DISTINCT number of `order_date`s each `customer_id` visited the restaurant. 
- ORDER BY the `customer_id` for a clean output. 

Results: 
- Customer A visited 4 times. 
- Customer B visited 6 times. 
- Customer C visited 2 times. 

In [7]:
%%sql 

select customer_id, count(distinct order_date) as visited_count
from sales 
group by customer_id
order by customer_id

 * sqlite:////Users/averylee/Desktop/DS/SQL/dannys_diner/dannys_diner.db
Done.


customer_id,visited_count
A,4
B,6
C,2


<a id="question-3"></a>
#### 3. What was the first item from the menu purchased by each customer?

Steps: 
- Rank the order each item was purchased for each customer. Use `DENSE_RANK` so that multiple orders are ranked the same if they were purchased on the same day by that customer. 
- Get only the DISTINCT menu items that were ordered on the first day that customer visited the restaurant. 

Results: 
- Customer A ordered both sushi and curry during their first visit. 
- Customer B ordered curry during their first visit. 
- Customer C ordered ramen during their first visit. 

In [8]:
%%sql 

with first_orders as (
    select 
        s.customer_id, 
        s.order_date, 
        m.product_name, 
        dense_rank() over (
            partition by s.customer_id 
            order by s.order_date
        ) as rank 
    from sales s join menu m on s.product_id = m.product_id
)

select distinct customer_id, order_date, product_name
from first_orders 
where rank = 1

 * sqlite:////Users/averylee/Desktop/DS/SQL/dannys_diner/dannys_diner.db
Done.


customer_id,order_date,product_name
A,2021-01-01,sushi
A,2021-01-01,curry
B,2021-01-01,curry
C,2021-01-01,ramen


<a id="question-4"></a>
#### 4. What is the most purchased item on the menu and how many times was it purchased by all customers?

Steps: 
- COUNT the number of times each menu item was purchased. 
- ORDER BY each count in descending order to get the most purchased item at the top. 

Results: 
- The ramen was the most purchased item with a total of 8 orders. Curry and sushi come next respectively with a count of 4 and 3 orders. 

In [9]:
%%sql 

select m.product_name, count(m.product_name) as count
from sales s join menu m on s.product_id = m.product_id
group by m.product_name
order by count desc

 * sqlite:////Users/averylee/Desktop/DS/SQL/dannys_diner/dannys_diner.db
Done.


product_name,count
ramen,8
curry,4
sushi,3


<a id="question-5"></a>
#### 5. Which item was the most popular for each customer?

Steps: 
- For each customer, COUNT the number of times they purchased each item. 
- DENSE_RANK this count in descending order. This means the most ordered item will be 1st, and the least ordered item will be last. 
- Get the items that were ordered most for each customer. 

Results: 
- Customer A ordered ramen the most number of times, with a total of 3 orders. 
- Customer B ordered sushi, ramen, and curry the same number of times, each being 2. 
- Customer C ordered ramen the most number of times, with a total of 3 orders. 

In [10]:
%%sql 

with most_purchased as (
    select 
        s.customer_id, 
        m.product_name, 
        count(m.product_name) as count, 
        dense_rank() over (
            partition by s.customer_id 
            order by count(m.product_name) desc
        ) as rank
    from sales s join menu m on s.product_id = m.product_id 
    group by s.customer_id, m.product_name
    order by s.customer_id, rank
) 

select customer_id, product_name, count
from most_purchased 
where rank = 1

 * sqlite:////Users/averylee/Desktop/DS/SQL/dannys_diner/dannys_diner.db
Done.


customer_id,product_name,count
A,ramen,3
B,sushi,2
B,ramen,2
B,curry,2
C,ramen,3


<a id="question-6"></a>
#### 6. Which item was purchased first by the customer after they became a member?

Steps: 
- INNER JOIN `sales` with `members` gets only the orders for customers that have joined the loyalty program, which is what we want for this question. 
- For each customer, get only the orders that were placed strictly after their `join_date`. 
- RANK the menu items by `order_date` for each customer after they joined the loyalty program. 
- Get the menu items that were purchased first after they became a member. 

Results: 
- Customer A purchased ramen first after becoming a member. 
- Customer B purchased sushi first after becoming a member. 
- Customer C never became a member. 

In [11]:
%%sql 

with orders_after_join as (
    select 
        s.customer_id, 
        s.order_date, 
        mem.join_date, 
        m.product_name, 
        dense_rank() over (
            partition by s.customer_id 
            order by s.order_date
        ) as rank
    from sales s join members mem on s.customer_id = mem.customer_id join menu m on s.product_id = m.product_id 
    where s.order_date > mem.join_date
    order by s.customer_id, s.order_date, m.product_name
)

select customer_id, product_name
from orders_after_join 
where rank = 1

 * sqlite:////Users/averylee/Desktop/DS/SQL/dannys_diner/dannys_diner.db
Done.


customer_id,product_name
A,ramen
B,sushi


<a id="question-7"></a>
#### 7. Which item was purchased just before the customer became a member?

This question is similar to the previous question, with just a few tweaks. 

Steps: 
- INNER JOIN `sales` with `members` gets only the orders for customers that have joined the loyalty program, which is what we want for this question. 
- For each customer, get only the orders that were placed strictly before their `join_date`. 
- RANK the menu items by `order_date` in descending order for each customer before they joined the loyalty program. This means the first date before they joined the program will be ranked 1. 
- Get the menu items that were purchased just before they became a member. 

Results: 
- Customer A purchased curry and sushi just before becoming a member. 
- Customer B purchased sushi just before becoming a member. 
- Customer C never became a member. 

In [12]:
%%sql 

with orders_before_join as (
    select 
        s.customer_id, 
        s.order_date, 
        mem.join_date, 
        m.product_name, 
        dense_rank() over (
            partition by s.customer_id 
            order by s.order_date desc
        ) as rank
    from sales s join members mem on s.customer_id = mem.customer_id join menu m on s.product_id = m.product_id 
    where s.order_date < mem.join_date
    order by s.customer_id, s.order_date, m.product_name
)

select customer_id, product_name
from orders_before_join 
where rank = 1

 * sqlite:////Users/averylee/Desktop/DS/SQL/dannys_diner/dannys_diner.db
Done.


customer_id,product_name
A,curry
A,sushi
B,sushi


<a id="question-8"></a>
#### 8. What is the total items and amount spent for each member before they became a member?

Steps: 
- INNER JOIN `sales` with `members` to only get customers who have become members. JOIN with `menu` to get the price of each menu item. 
- Keep only the orders WHERE it was placed strictly before the customer joined as a member. 
- GROUP BY each `customer_id` to get the COUNT of purchased items and SUM of the total amount spent. 

Results: 
- Customer A purchased a total of 2 items, spending $25 before becoming a member. 
- Customer B purchased a total of 3 items, spending $40 before becoming a member. 

In [13]:
%%sql 

select s.customer_id, count(m.product_name) as count, sum(m.price) as total_price
from sales s join members mem on s.customer_id = mem.customer_id join menu m on s.product_id = m.product_id
where s.order_date < mem.join_date
group by s.customer_id
order by s.customer_id

 * sqlite:////Users/averylee/Desktop/DS/SQL/dannys_diner/dannys_diner.db
Done.


customer_id,count,total_price
A,2,25
B,3,40


<a id="question-9"></a>
#### 9. If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?

Steps: 
- JOIN `sales` and `menu` to get the price for each item a customer has purchased. 
- Use CASE and END to determine the number of points each purchase gets WHEN an order is sushi or not. 
- SUM the total points received for each customer. 

Results: 
- Customer A has a total of 860 points. 
- Customer B has a total of 940 points. 
- Customer C has a total of 360 points. 

In [14]:
%%sql 

with customer_order_points as (
    select 
        s.customer_id, 
        m.product_name, 
        m.price, 
        case
            when m.product_name = 'sushi' then m.price * 10 * 2
            else m.price * 10
        end as points 
    from sales s join menu m on s.product_id = m.product_id
)

select customer_id, sum(points) as points
from customer_order_points 
group by customer_id
order by customer_id 

 * sqlite:////Users/averylee/Desktop/DS/SQL/dannys_diner/dannys_diner.db
Done.


customer_id,points
A,860
B,940
C,360


<a id="question-10"></a>
#### 10. In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?

Steps: 
- INNER JOIN `sales` with `members` to only get the customers who are members. JOIN again with `menu` to get the price of each item. 
- The summarized points calculation is as follows: if the order is sushi (at any time) OR the order was placed within the first week of the `join_date` inclusive of the first day, points is x20 the price. Otherwise, it is x10 the price. 
- SUM all the points for each customer WHERE the `order_date` is before the end of January. 

Results: 
- By 2021-01-31, Customer A has 1370 points. 
- By 2021-01-31, Customer B has 820 points. 

In [15]:
%%sql 
with points_week_after_join as (
    select s.customer_id, s.order_date, mem.join_date, m.product_name, m.price, case 
    when m.product_name = 'sushi' or (s.order_date >= mem.join_date and s.order_date < date(mem.join_date, '+7 day')) then m.price * 10 * 2 -- first week after becoming member 
    else m.price * 10
    end as points 
    from sales s join members mem on s.customer_id = mem.customer_id join menu m on s.product_id = m.product_id 
)

select customer_id, sum(points) as points
from points_week_after_join
where order_date <= '2021-01-31'
group by customer_id
order by customer_id

 * sqlite:////Users/averylee/Desktop/DS/SQL/dannys_diner/dannys_diner.db
Done.


customer_id,points
A,1370
B,820


## Bonus Tasks

<a id="bonus-1"></a>
#### 1. Create a table that contains `customer_id`, `order_date`, `product_name`, `price`, and `member` which indicates 'Y' if a customer is a member on that date and 'N' if they are not. 

Steps: 
- JOIN `sales` and `menu` to get the `price` for all items ordered. 
- Determine if a customer is a member at that point in time by SELECTing the `join_date` for that particular customer. 

In [16]:
%%sql 

drop table if exists sales_with_member_info; 

create table sales_with_member_info as 
select 
    s.customer_id, 
    s.order_date, 
    m.product_name, 
    m.price, 
    case 
        when s.order_date >= (select distinct join_date from members mem where mem.customer_id = s.customer_id) then 'Y'
        else 'N'
    end as member
from sales s join menu m on s.product_id = m.product_id; 

select * 
from sales_with_member_info

 * sqlite:////Users/averylee/Desktop/DS/SQL/dannys_diner/dannys_diner.db
Done.
Done.
Done.


customer_id,order_date,product_name,price,member
A,2021-01-01,sushi,10,N
A,2021-01-01,curry,15,N
A,2021-01-07,curry,15,Y
A,2021-01-10,ramen,12,Y
A,2021-01-11,ramen,12,Y
A,2021-01-11,ramen,12,Y
B,2021-01-01,curry,15,N
B,2021-01-02,curry,15,N
B,2021-01-04,sushi,10,N
B,2021-01-11,sushi,10,Y


<a id="bonus-2"></a>
#### 2. Using the saved table above, also add `ranking` which indicates the order of each item for a customer after they have joined the loyalty program. 

Steps: 
- Use the table created in the previous task. 
- Only RANK for after the customer joined the loyalty program and ORDER BY the `order_date'`

In [17]:
%%sql 

drop table if exists ranking_info_after_member; 

create table ranking_info_after_member as 
select *, case
when member = 'N' then NULL
else dense_rank() over (partition by customer_id, member order by order_date)
end as ranking
from sales_with_member_info; 

select * 
from ranking_info_after_member

 * sqlite:////Users/averylee/Desktop/DS/SQL/dannys_diner/dannys_diner.db
Done.
Done.
Done.


customer_id,order_date,product_name,price,member,ranking
A,2021-01-01,sushi,10,N,None
A,2021-01-01,curry,15,N,None
A,2021-01-07,curry,15,Y,1
A,2021-01-10,ramen,12,Y,2
A,2021-01-11,ramen,12,Y,3
A,2021-01-11,ramen,12,Y,3
B,2021-01-01,curry,15,N,None
B,2021-01-02,curry,15,N,None
B,2021-01-04,sushi,10,N,None
B,2021-01-11,sushi,10,Y,1


#